In [1]:
%matplotlib inline
import torch
from torch import nn
from d2l import torch as d2l
from d2l import torch as d2l
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

In [2]:
# -------------------------------
# 1. 定义预处理（将图像转为张量）
# -------------------------------
transform = transforms.ToTensor()

In [3]:
# -------------------------------
# 2. 下载并加载训练数据
# -------------------------------
train_data = datasets.FashionMNIST(
    root="./data", train=True, transform=transform, download=False
)
train_iter = DataLoader(train_data, batch_size=18, shuffle=True)

RuntimeError: Dataset not found. You can use download=True to download it

In [ ]:
# -------------------------------
# 3. 显示一批图像和标签
# -------------------------------
def show_images(images, labels, classes, rows=2, cols=9):
    fig, axes = plt.subplots(rows, cols, figsize=(cols * 1.5, rows * 1.5))
    for i in range(rows * cols):
        ax = axes[i // cols, i % cols]
        ax.imshow(images[i].squeeze(), cmap='gray')
        ax.set_title(classes[labels[i]])
        ax.axis('off')
    plt.tight_layout()
    plt.show()

In [ ]:
# -------------------------------
# 4. 获取一批数据并显示
# -------------------------------
class_names = [
    'T-shirt', 'Trouser', 'Pullover', 'Dress', 'Coat',
    'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot'
]

In [ ]:
# 从 train_loader 中取出一批数据
images, labels = next(iter(train_iter))
show_images(images, labels, class_names)

In [ ]:
# 定义 softmax 回归模型
class SoftmaxRegression(nn.Module):
    def __init__(self, num_inputs, num_outputs):
        super().__init__()
        self.linear = nn.Linear(num_inputs, num_outputs)  # 线性层 y = XW + b

    def forward(self, X):
        return self.linear(X)  # 输出 logits（未经过 softmax）

In [ ]:
net = SoftmaxRegression(784, 10)

for name, param in net.named_parameters():
    print(f"{name}: shape={param.shape}")

In [ ]:
def softmax(X):
    X_exp = torch.exp(X)                      # 每个元素做 e^x，放大差异
    partition = X_exp.sum(dim=1, keepdim=True)  # 每一行求和，作为分母
    return X_exp / partition                  # 每一行除以总和，输出概率分布

In [ ]:
X = torch.normal(0, 1, (2, 784))     # 构造一个 batch 的输入（2 张图，每张 28x28 展平）
net = SoftmaxRegression(784, 10)     # 定义模型
logits = net(X)                      # 得到每张图对10类的打分（logits）
probs = softmax(logits)              # 对 logits 做 softmax 得到概率分布

print(probs)
print(probs.sum(dim=1))              # 每行（每张图）的概率和应该是1

In [ ]:
def cross_entropy(y_hat, y):
    return -torch.log(y_hat[range(len(y_hat)), y])

In [ ]:
def accuracy(y_hat, y):
    if len(y_hat.shape) > 1 and y_hat.shape[1] > 1:
        y_hat = y_hat.argmax(dim=1)  # 如果是概率分布，就取最大概率对应的类别索引
    cmp = y_hat.type(y.dtype) == y   # 比较预测类别和真实标签是否一致
    return float(cmp.type(y.dtype).sum())  # 转为数字，统计预测正确的数量

In [ ]:
net = SoftmaxRegression(784, 10)                  # 定义模型
loss = nn.CrossEntropyLoss()
trainer = torch.optim.SGD(net.parameters(), lr=0.1)

num_epochs = 10                                   # 训练轮数
for epoch in range(num_epochs):
    train_loss, train_acc, n = 0.0, 0.0, 0
    for X, y in train_iter:                       # 遍历所有小批量
        X = X.reshape(X.shape[0], -1)
        y_hat = net(X)                            # 前向传播：计算 logits
        l = loss(y_hat, y)
        trainer.zero_grad()                       # 梯度清零
        l.backward()                              # 反向传播
        trainer.step()                            # 参数更新

        train_loss += float(l)                    # 累加损失
        train_acc += accuracy(y_hat, y)           # 累加正确数量
        n += y.shape[0]                           # 样本总数

    print(f'epoch {epoch + 1}, loss {train_loss / n:.3f}, acc {train_acc / n:.3f}')

In [ ]:
test_data = datasets.FashionMNIST(
    root="./data", train=False, transform=transforms.ToTensor(), download=False
)
test_iter = DataLoader(test_data, batch_size=18, shuffle=False)

In [ ]:
def predict(net, test_iter, n=18):
    for X, y in test_iter:
        X = X.reshape(X.shape[0], -1)
        y_hat = net(X)
        preds = y_hat.argmax(dim=1)
        return X.reshape((-1, 1, 28, 28)), y, preds  # 返回图像、真实、预测

# 获取图像、真实标签、预测标签
images, true_labels, pred_labels = predict(net, test_iter)

# 显示
show_images(images, pred_labels, class_names)
print("真实标签：", [class_names[i] for i in true_labels])
print("预测标签：", [class_names[i] for i in pred_labels])
